In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# A$^*$ Search

The module `Set` implements <em style="color:blue;">sets</em> as 
<a href="https://en.wikipedia.org/wiki/AVL_tree">AVL trees</a>.
The API provided by `Set` offers the following functions and methods:
- `Set()` creates an empty set.
- `S.isEmpty()` checks whether the set `S`is empty.
- `S.member(x)` checks whether `x` is an element of the set `S`.
- `S.insert(x)` inserts `x` into the set `S`.
  This does not return a new set but rather modifies the set `S`.
- `S.delete(x)` deletes `x` from the set `S`.
  This does not return a new set but rather modifies the set `S`.
- `S.pop()` returns the smallest element of the set `S`.
  Furthermore, this element is removed from `S`.
Since sets are implemented as ordered binary trees, the elements of a set need to be comparable, i.e. if `x` and `y` are inserted into a set, then the 
expression `x < y` must return a Boolean value and `<` has to define 
linear order.

The module `Set` can be used to implement a priority queue that supports the removal of elements.  All operation have a  complexity that is at most logarithmic in the number of elements stored in the `Set`.

In [2]:
import sys
sys.path.append('..')
import Set

The function `search` takes three arguments to solve a *search problem*:
- `start` is the start state of the search problem,
- `goal`is the goal state, and
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
  For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.
- `heuristic` is a function that takes two states as arguments.  It returns an estimate of the 
  length of the shortest path between these states.
  
If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$

The function `search` implements [A$^*$ search](https://en.wikipedia.org/wiki/A*_search_algorithm).  It uses the following data structures:
* `Parent` is a dictionary needed to compute the path.  If `Parent[ns] = s`, then we have `ns ∈ next_states(s)`.
* `Distance` is a dictionary.  For every state `s` that has been encountered, `Distance[s]` is the distance of `s` from `start`.
* `Estimate` is a dictionary. For every state `s` that has been encountered, `Estimate[s] = Distance[s] + heuristic(s, goal)`. 
   This dictionary is not strictly necessary, but when we have it, we are guaranteed that for any given state `s`
   the expression `heuristic(s, goal)` is computed at most once.
* `Frontier` is a priority queue that is implemented as an *ordered binary tree* via the module `Set`.  
   It stores pairs of the form `(d, s)` where `s` is a state, while `d` is an estimate of the length of 
   a path that leads from `start` to `goal` and passes through `s`.  Pairs with a small value of `d` have a high priority.

In [3]:
def search(start, goal, next_states, heuristic):
    Parent   = { start:start }
    Distance = { start: 0 }           
    estGoal  = heuristic(start, goal)
    Estimate = { start: estGoal }
    Frontier = Set.Set()
    Frontier.insert( (estGoal, start) )
    while not Frontier.isEmpty():
        estimate, state = Frontier.pop()
        if state == goal:
            return path_to(state, Parent)
        stateDist = Distance[state]
        for ns in next_states(state):
            oldEstimate = Estimate.get(ns, None)
            newEstimate = stateDist + 1 + heuristic(ns, goal)
            if oldEstimate == None or newEstimate < oldEstimate:
                Distance[ns] = stateDist + 1
                Estimate[ns] = newEstimate
                Parent[ns]   = state
                Frontier.insert( (newEstimate, ns) )
                if oldEstimate != None:
                    Frontier.delete( (oldEstimate, ns) )

Given a `state` and a parent dictionary `Parent`, the function `path_to` returns a path leading to the given `state`.

In [4]:
def path_to(state, Parent):
    p = Parent[state]
    if p == state:
        return [state]
    return path_to(p, Parent) + [state]

Lets draw the start state and animate the solution that has been found.

In [5]:
%run Sliding-Puzzle.ipynb

In [6]:
%load_ext memory_profiler

Solving the 8-puzzle takes less than half a second and uses 6 megabytes.

In [9]:
%%time
%memit Path = search(start, goal, next_states, manhattan)
print(len(Path)-1)

peak memory: 72.44 MiB, increment: 0.44 MiB
31
Wall time: 732 ms


In [8]:
animation(Path)

Canvas(height=300, width=300)

Solving the 15-puzzle takes less than 8 seconds and uses 93 megabytes.

In [10]:
%%time
%memit Path = search(start2, goal2, next_states, manhattan)
print(len(Path)-1)

peak memory: 141.40 MiB, increment: 69.96 MiB
36
Wall time: 5 s


In [ ]:
animation(Path)